In [1]:
from src.datasetSimulation.TFFamilyClass import TfFamily
from src.datasetSimulation.simulate_data import SimulatedData
from src.utils.preProcessing import *
from src.modelTraining.maTransformer import *
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from scipy.signal import fftconvolve
from torch.nn.utils.rnn import pad_sequence
import re
import numpy as np
import matplotlib.pyplot as plt
import random
import torch
import torch.nn as nn
import torch.optim as optim

import time
import sys


from copy import deepcopy

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
tf_object = TfFamily("data/raw_data/PWM.txt", "data/raw_data/prot_seq.txt")

In [4]:
data = SimulatedData(tf_object, n=1)

/Users/mgrapotte/LabWork/LearnTF/src/datasetSimulation/TFFamilyClass.py:71: RuntimeWarning: divide by zero encountered in log2
  m = np.log2(ppm.T/bg)


In [5]:
data.dummy_data

,id,prot_seq,ppm,dna_seq,label
0,prot_00seq_0,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
1,prot_00seq_1,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
2,prot_00seq_2,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
3,prot_00seq_3,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
4,prot_00seq_4,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
...,...,...,...,...,...
395,prot_11seq_95,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1
396,prot_11seq_96,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1
397,prot_11seq_97,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1
398,prot_11seq_98,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1


In [6]:
prot_seq = data.dummy_data.prot_seq.iloc[0]
dna_seq = data.dummy_data.dna_seq.iloc[0]

In [7]:
prot_encoded = encode_protein(prot_seq)

In [8]:
dna_encoded = encode_dna(dna_seq)

In [9]:
print(prot_encoded.shape)
print(dna_encoded.shape)

(21, 57)
(4, 100)


In [10]:
prot_tensor = torch.from_numpy(prot_encoded).reshape(1,1,prot_encoded.shape[0], prot_encoded.shape[1]).float()

In [11]:
dna_tensor = torch.from_numpy(dna_encoded).reshape(1,1,dna_encoded.shape[0], dna_encoded.shape[1]).float()

In [12]:
model = maTransformerBlock(filter_dna_size=5, filter_protein_size=5, nb_heads=1)

In [13]:
stacked_prot_tensor = torch.stack([prot_tensor, prot_tensor]).reshape(2,1,21,57)
stacked_dna_tensor = torch.stack([dna_tensor, dna_tensor]).reshape(2,1,4,100)

In [14]:
stacked_prot_tensor.shape

torch.Size([2, 1, 21, 57])

In [15]:
model(stacked_dna_tensor, stacked_prot_tensor)

tensor([[[-0.1126,  0.0546]],

        [[-0.1177,  0.0577]]], grad_fn=<AddBackward0>)

In [16]:
from torch.nn.functional import relu

In [17]:
import torch.nn.functional as F

In [263]:
F.relu(dna_tensor)

tensor([[[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

### defining the data loader

In [264]:
from torch.utils.data import Dataset
encode

NameError: name 'encode' is not defined

In [265]:
data.dummy_data.columns

Index(['id', 'prot_seq', 'ppm', 'dna_seq', 'label'], dtype='object')

In [266]:
data.dummy_data

,id,prot_seq,ppm,dna_seq,label
0,prot_00seq_0,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
1,prot_00seq_1,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
2,prot_00seq_2,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
3,prot_00seq_3,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
4,prot_00seq_4,RGPRTTIKAKQLETLKXXXXXAAFAATPKPTRHIREQLAQETGLNM...,"[[1.0, 1.0, 1.0, 1.0, 1.0], [0.0, 0.0, 0.0, 0....",TTTTTTTTTTTTTTTTTTTTTTTTTAAAAATTTTTTTTTTTTTTTT...,1
...,...,...,...,...,...
395,prot_11seq_95,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1
396,prot_11seq_96,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1
397,prot_11seq_97,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1
398,prot_11seq_98,RGPRTTIKAKQLETLKAAFAATPKPTRHIREQLAQETGLNMRVIYY...,"[[0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1....",TTTTTTTTTTTTTTTTTTTTTTTTTCCCCCTTTTTTTTTTTTTTTT...,1


In [267]:
class generalData(Dataset):
    def __init__(self, data):
        """
        data is the pandas dataframe from the xxx class
        """
        
        self.dna = np.stack([encode_dna(d) for d in data["dna_seq"]])
        self.dna = torch.from_numpy(self.dna).reshape(self.dna.shape[0],1,self.dna.shape[1],self.dna.shape[2]).float()
        self.prot = np.stack([encode_protein(p) for p in data["prot_seq"]])
        self.prot = torch.from_numpy(self.prot).reshape(self.prot.shape[0],1,self.prot.shape[1],self.prot.shape[2]).float()
        #self.label = torch.from_numpy(OneHotEncoder().fit_transform(data["label"].values.reshape(-1,1)).toarray()).float()
        self.label = data["label"].values
    def __len__(self):
        return len(self.label)
        
    def __getitem__(self, idx):
        #return self.dna[idx,:,:,:], self.prot[idx,:,:,:], self.label[idx,:]
        return self.dna[idx,:,:,:], self.prot[idx,:,:,:], self.label[idx]

In [268]:
from torch.utils.data import DataLoader

In [269]:
test = generalData(data.dummy_data)

In [270]:
batch_size = 20

In [271]:
trainloader = DataLoader(generalData(data.dummy_data), batch_size=batch_size, shuffle=True)

In [272]:
dna, prot, labels = next(iter(loaded))

In [273]:
dna.shape

torch.Size([20, 1, 4, 100])

In [274]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01, betas=(0.9, 0.999))
model = maTransformerBlock(filter_dna_size=5, filter_protein_size=5, nb_heads=1)

In [279]:
test_outputs = []
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        dna, prot, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(dna, prot).squeeze()
        test_outputs.append(outputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        #running_loss += loss.item()
        #print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')

print('Finished Training')

Finished Training


In [280]:
test_outputs

[tensor([[ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0583, -0.0099],
         [ 0.0977, -0.0106],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411]], grad_fn=<SqueezeBackward0>),
 tensor([[ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0509, -0.0021],
         [ 0.0991, -0.0236],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.0875, -0.0411],
         [ 0.

In [240]:
dna = test.__getitem__(2)[0]
prot = test.__getitem__(2)[1]
label = test.__getitem__(2)[2]

In [241]:
prot.shape

torch.Size([1, 21, 57])

In [242]:
model(dna.reshape(1,1,4,100), prot.reshape(1,1,21,57), verbose=True)

dna_conv: tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.1278, 0.3853, 0.4241, 0.0917, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
           0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]]],
       grad_fn=<ReluBackward0>)
prot_conv: tensor([[[[0.0726, 0.2303, 0.0782, 0.1085, 

tensor([[0.1574, 0.2417]], grad_fn=<AddmmBackward>)

In [237]:
model.linear.weight.data

tensor([[-0.2054, -0.2519,  0.2580,  0.2576,  0.0727, -0.0370,  0.0609,  0.2849,
          0.0681,  0.1212],
        [-0.1528,  0.2504,  0.2408, -0.0699,  0.2250, -0.0814, -0.2259,  0.0356,
         -0.1518, -0.0424]])

In [223]:
print(dna)

tensor([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,


In [245]:
test_outputs

[tensor([[ 0.2241, -0.1771],
         [ 0.2217, -0.1703],
         [ 0.2228, -0.1737],
         [ 0.2217, -0.1704],
         [ 0.2220, -0.1712],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2193, -0.1638],
         [ 0.2135, -0.1473],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2241, -0.1771],
         [ 0.2197, -0.1647],
         [ 0.2219, -0.1709]], grad_fn=<AddmmBackward>),
 tensor([[ 0.2217, -0.1703],
         [ 0.2229, -0.1738],
         [ 0.2252, -0.1802],
         [ 0.2252, -0.1802],
         [ 0.2241, -0.1771],
         [ 0.2186, -0.1616],
         [ 0.2185, -0.1614],
         [ 0.2220, -0.1711],
         [ 0.2197, -0.1645],
         [ 0.2171, -0.1571],
         [ 0.2191, -0.1629],
         [ 0.2166, -0.1558],
         [ 0.2197, -0.1647],
         [ 0.225

### Creating a new model architecture and setting the weights

In [18]:
model = maTransformerBlock(filter_dna_size=5, filter_protein_size=5, nb_heads=1)

In [19]:
model.conv_dna.weight = torch.nn.parameter(torch.from_numpy(np.array([[]])))

Parameter containing:
tensor([[[[ 0.0931,  0.1964,  0.0025,  0.1341, -0.1125],
          [ 0.2211,  0.0263, -0.0962, -0.1863,  0.1535],
          [-0.0334, -0.1519, -0.2133, -0.1493,  0.0541],
          [-0.1900,  0.0433, -0.2075, -0.1428,  0.0273]]]], requires_grad=True)

In [ ]:
torch.from_numpy(np.array)